In [ ]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/ndcbe/CBE60499/main/notebooks/helper.py"
    import helper
    helper.install_glpk ()
    helper.install_idaes()
    helper.install_ipopt()
!apt-get install -y coinor-cbc

In [1]:
import pandas as pd

df_clients = pd.read_csv('./content/base/clients.csv')
df_depots = pd.read_csv('./content/base/depots.csv')
df_vehicles = pd.read_csv('./content/base/vehicles.csv')

In [2]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [3]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C


print("model.D =", list(model.D.data()))
print("model.C =", list(model.C.data()))
print("model.V =", list(model.V.data()))
print("model.N =", list(model.N.data()))
print((lugares))

model.D = ['CD1']
model.C = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
model.V = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8']
model.N = ['CD1', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24']
[(-74.153536, 4.743359), (-74.09893796560621, 4.59795431125545), (-74.07557103763986, 4.687820646838871), (-74.10708524062704, 4.70949446000624), (-74.09727965657427, 4.605029068682624), (-74.16464148202755, 4.648463876533332), (-74.12083799988112, 4.662137416953968), (-74.02213076607309, 4.697499030379109), (-74.17207549744595, 4.649416884236942), (-74.15615257246444, 4.606310650273935), (-74.09041145358674, 4.557379705282216), (-74.17802255204528, 4.591594072172954), (-74.1015410917749, 4.7564172406324055), (-74.09690889182339, 4.646217006050524), (-74.1219200708342, 

In [4]:
import openrouteservice
from openrouteservice import convert
import json

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)

matrix = client.distance_matrix(
    locations=lugares,
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

In [5]:
precio_km = 3032.1
cost = {}
distance = {}

dist_matrix = matrix['distances']
print(dist_matrix)
print(len(dist_matrix))

for i,i1 in zip(range(len(lugares)),  list(model.N.data())):
        for j,j1 in zip(range(len(lugares)),  list(model.N.data())):
            if i != j:
                distancia_km = dist_matrix[i][j]
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio
print(distance)
print(cost)

[[0.0, 27.19, 16.99, 12.37, 26.7, 22.64, 19.65, 25.02, 23.89, 30.67, 33.86, 33.49, 15.87, 21.66, 10.36, 26.79, 33.91, 26.61, 19.87, 20.93, 13.14, 27.84, 24.41, 11.34, 30.87], [30.96, 0.0, 14.29, 19.44, 1.13, 12.5, 11.32, 17.81, 13.75, 7.51, 7.39, 10.33, 25.81, 8.3, 20.31, 1.29, 7.44, 4.49, 9.84, 14.97, 23.09, 21.68, 5.38, 21.29, 4.89], [18.34, 12.69, 0.0, 6.83, 12.2, 14.3, 7.43, 9.22, 15.55, 15.46, 19.36, 18.27, 13.19, 6.44, 7.69, 12.29, 19.41, 11.39, 8.78, 5.6, 10.47, 12.8, 9.91, 8.67, 15.81], [13.76, 15.97, 5.77, 0.0, 15.48, 11.42, 8.43, 13.8, 12.67, 19.45, 22.64, 22.27, 8.62, 10.44, 3.11, 15.57, 22.69, 15.39, 8.65, 9.71, 5.89, 16.61, 13.19, 4.09, 19.64], [29.95, 1.05, 13.28, 18.43, 0.0, 11.49, 10.31, 16.81, 12.74, 7.9, 8.32, 10.71, 24.8, 7.29, 19.3, 1.25, 8.37, 4.17, 8.83, 13.97, 22.08, 20.68, 4.38, 20.27, 5.27], [21.07, 13.4, 16.25, 17.53, 12.91, 0.0, 6.96, 22.65, 1.54, 7.17, 20.06, 9.56, 22.37, 13.33, 18.39, 13.0, 16.51, 10.38, 6.73, 18.56, 21.17, 25.46, 12.83, 19.37, 14.8], [23.0

In [6]:
depot_capacity = {}
for depot in df_depots['DepotID']:
    depot_capacity[f'CD{depot}'] = 9999999

print(depot_capacity)

demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c

print(demand)

vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c

print(vehicle_capacity)

vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r

print(vehicle_range)

{'CD1': 9999999}
{'C1': 13, 'C2': 15, 'C3': 12, 'C4': 15, 'C5': 20, 'C6': 17, 'C7': 17, 'C8': 20, 'C9': 20, 'C10': 15, 'C11': 17, 'C12': 12, 'C13': 21, 'C14': 15, 'C15': 17, 'C16': 10, 'C17': 25, 'C18': 12, 'C19': 11, 'C20': 15, 'C21': 14, 'C22': 18, 'C23': 15, 'C24': 11}
{'V1': 130, 'V2': 140, 'V3': 120, 'V4': 100, 'V5': 70, 'V6': 55, 'V7': 110, 'V8': 114}
{'V1': 170, 'V2': 200, 'V3': 180, 'V4': 90, 'V5': 100, 'V6': 170, 'V7': 150, 'V8': 140}


In [7]:
# Parámetros
model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def obj_rule(model):
    return sum(model.cost[i,j] * model.x[v,i,j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)

def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)

"""
def return_route_rule(model, v, d):
    return sum(model.x[v, i, d] for i in model.N if i != d) == model.y[v,d]

model.return_route = Constraint(model.V, model.D, rule=return_route_rule)
"""

def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distance[i,j] * model.x[v,i,j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

# Resolver
from amplpy import modules
#solver = SolverFactory('cbc')
#solver.options["seconds"] = 300  # máximo 5 minutos
solver_name = "gurobi"
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
solver.options['TimeLimit'] = 300  # 5 minutos de tiempo máximo
solver.options['MIPGap'] = 0.01  # Establecer un límite en el gap de optimalidad
results = solver.solve(model)

model.name="unknown";
    - termination condition: maxIterations
    - message from solver: Gurobi 12.0.1\x3a time limit, feasible solution;
      objective 1079943.057; 466029 simplex iterations; 20962 branching nodes;
      absmipgap=97668.7, relmipgap=0.0904387


In [23]:
def print_routes(model):
    for v in model.V:
        print(f"\n>>> Ruta del vehículo {v}:")
        
        # 1. Encontrar el depósito asignado al vehículo y su capacidad
        depot = None
        depot_cap = 0
        for d in model.D:
            if value(model.y[v, d]) > 0.5:
                depot = d
                depot_cap = depot_capacity[d]
                break
        
        if depot is None:
            print("Vehículo no asignado a ningún depósito.")
            continue
        
        print(f" - Depósito de salida: {depot} (Capacidad: {depot_cap})")
        print(f" - Capacidad del vehículo: {vehicle_capacity[v]}")
        
        # 2. Reconstruir la ruta y calcular carga
        current_node = depot
        route = [current_node]
        visited = set()
        total_load = 0
        deliveries = {}  # Diccionario para guardar entregas entre nodos
        
        while True:
            next_node = None
            for j in model.N:
                if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                    next_node = j
                    break
            
            if next_node is None or next_node in visited:
                break
            
            # Registrar entrega si es a un cliente
            if next_node in model.C:
                delivery = demand[next_node]
                total_load += delivery
                deliveries[(current_node, next_node)] = delivery
            
            route.append(next_node)
            visited.add(next_node)
            current_node = next_node
        
        # 3. Imprimir resultados
        if len(route) > 1:
            print(f" - Ruta: {' → '.join(str(node) for node in route)}")
            print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]}")
            
            # Imprimir entregas entre pares de nodos
            print(" - Entregas entre nodos:")
            for (i, j), qty in deliveries.items():
                print(f"    De {i} → {j}: Entrega {qty} unidades")
        else:
            print("Vehículo no utilizado.")

# Llamar a la función
print_routes(model)


>>> Ruta del vehículo V1:
 - Depósito de salida: CD1 (Capacidad: 9999999)
 - Capacidad del vehículo: 130
 - Ruta: CD1 → C2 → C19 → C7 → C21 → C12 → CD1
 - Carga total transportada: 69/130
 - Entregas entre nodos:
    De CD1 → C2: Entrega 15 unidades
    De C2 → C19: Entrega 11 unidades
    De C19 → C7: Entrega 17 unidades
    De C7 → C21: Entrega 14 unidades
    De C21 → C12: Entrega 12 unidades

>>> Ruta del vehículo V2:
 - Depósito de salida: CD1 (Capacidad: 9999999)
 - Capacidad del vehículo: 140
 - Ruta: CD1 → C6 → C17 → C24 → C16 → C10 → C1 → C15 → C4 → C18 → CD1
 - Carga total transportada: 135/140
 - Entregas entre nodos:
    De CD1 → C6: Entrega 17 unidades
    De C6 → C17: Entrega 25 unidades
    De C17 → C24: Entrega 11 unidades
    De C24 → C16: Entrega 10 unidades
    De C16 → C10: Entrega 15 unidades
    De C10 → C1: Entrega 13 unidades
    De C1 → C15: Entrega 17 unidades
    De C15 → C4: Entrega 15 unidades
    De C4 → C18: Entrega 12 unidades

>>> Ruta del vehículo V3:

In [16]:
import folium
import openrouteservice
from openrouteservice import convert
from pyomo.environ import value
import itertools

# Paleta de colores (puedes expandirla si tienes más vehículos)
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculo_a_color = dict(zip(model.V, itertools.cycle(colores)))


client = openrouteservice.Client(key=API_KEY)

m = folium.Map(location=[4.65, -74.1], zoom_start=11)

coords= lugares
labels= list(model.N.data())


In [17]:
for label, coord in zip(labels, coords):
    folium.Marker(location=(coord[1], coord[0]), popup=label).add_to(m)

# Diccionario de índice a coordenada
label_to_coord = dict(zip(labels, coords))

# Supón que tienes model.x[v, i, j] y model.V y model.N cargado
# Simulación: rutas como lista de tuplas (vehículo, origen, destino)
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))
print(rutas_optimas)


[('V1', 'CD1', 'C2'), ('V1', 'C2', 'C19'), ('V1', 'C7', 'C21'), ('V1', 'C12', 'CD1'), ('V1', 'C19', 'C7'), ('V1', 'C21', 'C12'), ('V2', 'CD1', 'C6'), ('V2', 'C1', 'C15'), ('V2', 'C4', 'C18'), ('V2', 'C6', 'C17'), ('V2', 'C10', 'C1'), ('V2', 'C15', 'C4'), ('V2', 'C16', 'C10'), ('V2', 'C17', 'C24'), ('V2', 'C18', 'CD1'), ('V2', 'C24', 'C16'), ('V3', 'CD1', 'C23'), ('V3', 'C23', 'CD1'), ('V4', 'CD1', 'C22'), ('V4', 'C5', 'CD1'), ('V4', 'C8', 'C5'), ('V4', 'C9', 'C11'), ('V4', 'C11', 'C8'), ('V4', 'C22', 'C9'), ('V5', 'CD1', 'C3'), ('V5', 'C3', 'CD1'), ('V6', 'CD1', 'C14'), ('V6', 'C14', 'CD1'), ('V7', 'CD1', 'C20'), ('V7', 'C20', 'CD1'), ('V8', 'CD1', 'C13'), ('V8', 'C13', 'CD1')]


In [18]:
for (v, i, j) in rutas_optimas:
    coord_i = label_to_coord[i]
    coord_j = label_to_coord[j]
    color = vehiculo_a_color[v]
    try:
        route = client.directions(
            coordinates=[coord_i, coord_j],
            profile='driving-car',
            format='geojson'
        )
        folium.GeoJson(
            route,
            name=f"Ruta {v}: {i}->{j}",
            tooltip=f"{v}: {i} → {j}",
            style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
        ).add_to(m)
    except Exception as e:
        print(f"Error en ruta {i} → {j}: {e}")
m